In [ ]:
import os
import Cell_BLAST as cb
import utils
os.environ["CUDA_VISIBLE_DEVICES"] = utils.pick_gpu_lowest_memory()
cb.config.RANDOM_SEED = 0
cb.config.N_JOBS = 4
fixed_model_kwargs = dict(
    latent_dim=10, cat_dim=20,
    epoch=500, patience=20
)

In [ ]:
cb.__version__

---
# Mouse
## Zeisel_2018

In [ ]:
zeisel_2018 = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Zeisel_2018/data.h5")
utils.peek(zeisel_2018, "build/nervous_system/Zeisel_2018")
zeisel_2018.obs.head()

In [ ]:
zeisel_2018.obs.dtypes

In [ ]:
opt_model_kwargs = dict(prob_module_kwargs=dict(lambda_reg=0.01))
zeisel_2018_model = cb.directi.fit_DIRECTi(
    zeisel_2018, zeisel_2018.uns["seurat_genes"],
    **fixed_model_kwargs, **opt_model_kwargs
)
zeisel_2018.latent = zeisel_2018_model.inference(zeisel_2018)

In [ ]:
ax = zeisel_2018.visualize_latent("cell_ontology_class", size=0.1, scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/nervous_system/Zeisel_2018/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = zeisel_2018.visualize_latent("cell_type1", size=0.1, scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/nervous_system/Zeisel_2018/cell_type1.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = zeisel_2018.visualize_latent("region", size=0.1, scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/nervous_system/Zeisel_2018/region.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = zeisel_2018.visualize_latent("donor", size=0.1, scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/nervous_system/Zeisel_2018/donor.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
zeisel_2018.write_dataset("build/nervous_system/Zeisel_2018/Zeisel_2018.h5")

In [ ]:
%%capture capio
zeisel_2018_models = [zeisel_2018_model]
opt_model_kwargs = dict(batch_effect="donor")
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    zeisel_2018_models.append(cb.directi.fit_DIRECTi(
        zeisel_2018, zeisel_2018.uns["seurat_genes"],
        **fixed_model_kwargs, **opt_model_kwargs,
        random_seed=i
    ))
zeisel_2018_blast = cb.blast.BLAST(zeisel_2018_models, zeisel_2018)
zeisel_2018_blast.save("build/nervous_system/Zeisel_2018")

In [ ]:
with open("build/nervous_system/Zeisel_2018/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/nervous_system/Zeisel_2018/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(zeisel_2018_blast, "build/nervous_system/Zeisel_2018")

In [ ]:
%%writefile build/nervous_system/Zeisel_2018/predictable.txt
cell_ontology_class
cell_type1
region